In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install -U accelerate

In [4]:
import torch
import numpy as np
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

In [ ]:
checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

In [ ]:
raw_datasets = load_dataset('tyqiangz/multilingual-sentiments', 'english')
def change_labels(example):
    if example['label'] == 0:
        example['label'] = 2
    elif example['label'] == 2:
        example['label'] = 0
    return example

#print(raw_datasets['train'][0:3])
# Применение функции изменения меток к датасету
raw_datasets = raw_datasets.map(change_labels)
raw_datasets['train'][0:3]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["text"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["text"])

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
training_args = TrainingArguments("test-trainer")

In [12]:


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load('tyqiangz/multilingual-sentiments', 'english')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()